# 一時開発用。

**参考** : [connpass APIリファレンス](https://connpass.com/about/api/)


In [1]:
import requests
import json
import datetime
import time
import random
import math
# 年、月の計算はこれが必要。datetimeでは出来ない。
from dateutil.relativedelta import relativedelta

In [3]:
#  一時的にWIPフォルダでやっているから、一個上のディレクトリに抜ける必要がある。
#  ちゃんと本体に組み込む時には不要。
import sys
sys.path.append('..')

# 自作
from settings import target

In [4]:
# エンドポイント
url = 'https://connpass.com/api/v1/event/'

In [5]:
# API検索条件 ####################################
# 開催場所
location = '東京都'
# 検索キーワード(or)
# 検索結果が500を超えた分は、結果を取得できない。
# キーワードをシャッフルして、検索結果数に偏りが内容にしようとする気休め。
search_words = target.SEARCH_WORDS
random.shuffle(search_words)
# 今月を含め、何ヶ月先まで検索するか
search_months = 3
# 一度に取得するデータ件数。100で良い。（最小1, 最大100。apiの仕様。）
nb_get_data = 100

# システム設定 ###################################
# 一度に検索するキーワード数（検索結果が多すぎるとエラー）。実績値で40程度。
# 小さすぎると検索回数が増える。一度、アクセス過多で運営から注意を受けているので気をつかう。
nb_search_words = min(math.ceil(len(search_words) / 2), 40)
# get失敗時に中断するか(boolean)
continue_on_error = False
# リクエスト連投で空けるべき時間（秒）。１秒以上。
request_sleep_sec = 2


In [6]:
# 検索対象月のリスト作成
today = datetime.date.today()
ym = [(today + relativedelta(months=i)).strftime('%Y%m')
      for i in range(search_months)]

In [7]:
def get_response(url, params):
    """
    リクエストを投げて、レスポンスを受け取る。
    リクエストは必ずこの関数を使用することで、sleepを保証する。
    """
    time.sleep(request_sleep_sec)
    return requests.get(url, params=params)

In [8]:
def get_search_result(start, batch_start):
    q = {'count': nb_get_data,   # 取得件数
         'start': start,         # 検索の開始位置。結果セットを複数回に分けて取得しないといけないので。
         'ym': ym ,              # イベント開催年月
         'keyword': location,    #  キーワード (AND)
         'keyword_or': target.SEARCH_WORDS[batch_start: batch_start + nb_search_words]  # キーワード (OR)
        }
    return get_response(url, params=q)

In [ ]:
today = datetime.datetime.today()
# 結果セット全体の中での処理開始位置
for batch_start in list(range(len(search_words)))[: : nb_search_words]:
    print('*' * 10, 'batch_start :', batch_start, '*' * 100)
    start = 1
    nb_all_result = sys.maxsize     # 後で更新
    while start < nb_all_result:
        r = get_search_result(start, batch_start)
        # エラーチェック
        if not r.ok:
            if continue_on_error:
                print('APIからのGETに失敗: [{}] {}'.format(r.status_code, r.reason))
                print('エラーでも中断しない設定なので、監視を続けます。')
                continue
            else:
                assert r.ok, 'APIからのGETに失敗: [{}] {}'.format(r.status_code, r.reason)
        
        # 大した処理ではないので、わかりやすさを優先して、リスト内包表記は使用しない
        for event in r.json()['events']:
            event_datetime = datetime.datetime.strptime(event['started_at'][:-6], '%Y-%m-%dT%H:%M:%S')
            if today <= event_datetime:
                print( event_datetime, event['title'], event['event_url'])

        start += int(r.json()['results_returned'])
        nb_all_result = int(r.json()['results_available'])   # 同じキーワードなら、連続して検索しても同じ


********** batch_start : 0 ****************************************************************************************************
2017-10-04 19:30:00 世界最高レベルの精度のAIチャットボットを開発・導入する方法【10/4開催】 https://spj.connpass.com/event/67197/
2017-10-07 10:00:00 【ハッカソン】エンジニアの食健康習慣をHACKする「健康ハッカソン」 https://supporterz-seminar.connpass.com/event/67297/
2017-10-07 12:30:00 .NET Conf 2017 Tokyo, Japan https://csugjp.connpass.com/event/66004/
2017-10-05 19:30:00 【サポーターズCoLab勉強会】カンヌ受賞者が語るクリエイティブ ☓ テクノロジーの実例や現場の裏側 https://supporterz-seminar.connpass.com/event/67458/
2017-10-04 19:45:00 【サポーターズCoLab勉強会】開発プロセスを改善したら、社内コミュニケーションも円滑になった話。 https://supporterz-seminar.connpass.com/event/66932/
2017-10-04 19:30:00 【サポーターズCoLab勉強会】実践Laravel https://supporterz-seminar.connpass.com/event/66841/
2017-10-03 19:45:00 【サポーターズCoLab勉強会】Docker入門 https://supporterz-seminar.connpass.com/event/67179/
2017-10-03 19:30:00 【サポーターズCoLab勉強会】サーバーレスアプリケーションのすすめ https://supporterz-seminar.connpass.com/event/67112/
2017-10-02 19:30:00 【サポーター

2017-10-05 19:30:00 法務＆知財系ライトニングトーク2017 ＜リーガルテック祭＞ https://connpass.com/event/65612/
2017-10-14 13:00:00 System Center User Group Japan 第17回 勉強会 https://connpass.com/event/67133/
2017-09-28 18:30:00 AITC ブロックチェーン勉強会(会員限定) https://aitc-seminar.connpass.com/event/66599/
2017-10-12 19:00:00 データサイエンスセミナー 【found IT project #9】 https://foundit-project.connpass.com/event/66468/
2017-10-05 19:20:00 500 Happy Hour #2 https://connpass.com/event/67018/
2017-10-08 10:00:00 PHPカンファレンス2017 https://phpcon.connpass.com/event/65943/
2017-10-11 19:00:00 3社合同反省会 https://lig.connpass.com/event/66033/
2017-10-04 09:30:00 Python3ではじめるシステムトレード　第5回連続セミナー　午前の部 https://kinyuzaimu.connpass.com/event/66762/
2017-10-04 13:30:00 Python3ではじめるシステムトレード　第5回連続セミナー 午後の部 https://kinyuzaimu.connpass.com/event/66761/
2017-10-15 13:30:00 Python3ではじめるシステムトレード　第6回連続セミナー 午後の部 https://kinyuzaimu.connpass.com/event/67162/
2017-10-15 09:30:00 Python3ではじめるシステムトレード　第6回連続セミナー　午前の部 https://kinyuzaimu.connpass.com/event/67161/
2017-09-